---
title: Corregistration of UAV mosaics
subject: Tutorial
subtitle: Notebook that shows a prototype method to automatically corregister airborne/UAV orthomomosaics
short_title: UAV corregistration
authors:
  - name: Héctor Nieto
    affiliations:
      - Instituto de Ciencias Agrarias, ICA
      - CSIC
    orcid: 0000-0003-4250-6424
    email: hector.nieto@ica.csic.es
  - name: Benjamin Mary
    affiliations:
      - Insituto de Ciencias Agrarias
      - CSIC
    orcid: 0000-0001-7199-2885
license: CC-BY-SA-4.0
keywords: UAV, TSEB, LST-NDVI space
---

# Summary
This interactive Jupyter Notebook has the objective of showing one prototype method that automates the search of Ground Control Points between two UAV mosaics.

# Instructions
Read carefully all the text and follow the instructions.

Once each section is read, run the jupyter code cell underneath (marked as `In []`) by clicking the icon `Run`, or pressing the keys SHIFT+ENTER of your keyboard. A graphical interface will then display, which allows you to interact with and perform the assigned tasks.

To start, please run the following cell to import all the packages required for this notebook. Once you run the cell below, an acknowledgement message, stating all libraries were correctly imported, should be printed on screen.

In [ ]:
%matplotlib inline
from pathlib import Path
from ipywidgets import interact, interactive, fixed, widgets
from IPython.display import display
import numpy as np
print("Libraries imported correctly, you can continue to the next cells")

# Corregistering orthomosaics
In the previous exercise we saw how important is having well collocated UAV mosaicks between the TIR and spectral cameras. Not only for accurately extracting the soil and canopy temperatures, but for later be able to robustly parse the different canopy tratis and radiometric temperatures in TSEB. 

Althouh both phototgrametric software and UAV avionics/payloads have evolve a lot in the last years, it is not uncommon that the different orthomosaics generated from the different payload cameras do not perfectly match. This would involve the manual tasks of adding Ground Control Points, either with targets placed on ground or by indentifying distinctive patters in each mosaick.

:::{important}
This task can be tedious and time consuming, thus it woulc preclude the use of UAV imager in operational and/or near-real-time services.
:::

# The [gcp_tools in GitHub airborne_tools package](https://github.com/hectornieto/airborne_tools/blob/5db17192e638c2745dea5d918b9dcaffd05a14cf/airborne_tools/gcp_tools.py#L18)
For that reason, we have developed a prototype that aims to automatize the corregistration of orthomosaics. This prototype aims to collocate an orthomosaic[^1] (from now on we will call it the `slave`) over another orthomosaic or image that is considered as `reference`.

[^1]: Or any other geospatial raster image such as satellite imagery.

:::{hint}
The `reference` image can be a well georreferenced orthomosaic such as an RGB UAV mosaic, an aerial orthoimage or any other dataset. Indeed the **scale/resolution between the `reference` and the `slave` mosaic to be collocated does not need to be the same**
:::

Basically we are using SIFT {cite:p}`https://doi.org/10.1023/B:VISI.0000029664.99615.94` algorithm to find relevant features in each image, followed by FLANN {cite:p}`https://doi.org/10.1109/TPAMI.2014.2321376` algorithm to evaluate the similarities of features between both images (reference and slave) and find feature mathing in both images, which will be the ones considered as potential Ground Control Points (`GCP`s).

:::{note}
:class:drowpdown
Indeed SIFT+FLANN are the typical algorithms that photogrammetric software uses to peform the matching and overlapping between snapshots, pior to building the photogrammetric point cloud.
:::

## Dataset
For this exercise we will use the test example of [airborne_tools package](https://github.com/hectornieto/airborne_tools). The data is already located at the [./input/UAV](./input/UAV) folder. It consists of a UAV flight over an experimental vineyard located near Madrid (Spain) that acquired (among others):
* A multispectral image ([](./input/UAV/Sequoia_vnir_20220916.tif)) with the following bands:

  1. Green
  2. Red
  3. Red-edge
  4. Near Infrared
* A thermal image ([](./input/UAV/tir_odm_20220916.tif))with temperatures in Kelvin, scaled by a factor of 100

We recommend you to open both images in QGIS to better browse them and be aware of the lack of corregistration between mosaics.

:::{hint}
You can use the cars/pannels in the upper part of the scene as reference.
:::

Even the UAV was equipped with a RTK differential GPS and Ground Control Points were place on ground, you can still see some some displacements between mosaics.

## Preprocess the mosaics
As an optional step we could do is to preprocess the mosaics. Either the `reference` or the `slave`. This could be usefull for multidimensional imagery such as multispectral/hyperspectral data, since SIFT algorithm only works for single-band grayscale or RGB pictures. 

For this case we will reduce the multispectral `reference` scene to a single band brightness image, by applying a Principal Component Analysis, and selecting the first PCA, since this is the one that will 

In [ ]:
from airborne_tools import image_preprocessing as img

# Set the input folder location and raw VNIR and output PCA single-band image
workdir = Path()
test_dir = workdir / "input" / "UAV"

# Set the path to the input LST and VNIR images
lst_file = test_dir / 'tir_odm_20220916.tif'
vnir_image = test_dir / 'Sequoia_vnir_20220916.tif'
pca_image = test_dir / 'Sequoia_vnir_20220916_PC1.tif'

# Set the nodata of the VNIR imager
no_data=4294967296

# We will use all the VNIR bands (4) to create the PCA image
vnir_bands = [0, 1, 2, 3]

# And we will save only the first PC band, considering that this component 
pca_components = 1
# We need to reduce the dimensionality of the master image to a single grayscale band.
# We therefore apply a PCA reduction to get a grayscale image combining all spectral bands
if not pca_image.exists():
    img.pca(vnir_image,
            no_data=no_data,
            use_bands=vnir_bands,
            pca_components=pca_components,
            outfile=pca_image,
            normalize=True)

print(f"Created VNIR PCA image at {pca_image}")

:::{seealso}
:class:dropdown
You can check the [PCA image reduction GitHub source code](https://github.com/hectornieto/airborne_tools/blob/5db17192e638c2745dea5d918b9dcaffd05a14cf/airborne_tools/image_preprocessing.py#L9)
:::

### Visualize both grayscale images
You can now visualized the grayscale images for both the `reference` VNIR PCA1 and the `slave` LST.

:::{note}
You can also use QGIS to better visualize both images and confirm the corregistration issues between mosaics.
:::

In [ ]:
from osgeo import gdal
from bokeh.plotting import *
from bokeh import palettes as pal
from bokeh.models.mappers import LinearColorMapper
from bokeh.io import output_notebook
from bokeh.resources import INLINE
output_notebook(resources=INLINE)

# Open and read the LST file
fid = gdal.Open(lst_file, gdal.GA_ReadOnly)
lst = fid.GetRasterBand(1).ReadAsArray().astype(float)
# Set LST NaN
lst_no_data = 65535
lst[lst == lst_no_data] = np.nan
# Open and read the NDVI file
fid = gdal.Open(pca_image, gdal.GA_ReadOnly)
pca = fid.GetRasterBand(1).ReadAsArray()
master_geo = fid.GetGeoTransform()
del fid

rows, cols = int(0.3 * lst.shape[0]), int(0.3 * lst.shape[1])
s1 = figure(title="LST", width=cols, height=rows, x_range=[0, cols], y_range=[0, rows])
s1.axis.visible = False
s1.image(image=[np.flipud(lst)], x=[0], y=[0], dw=cols, dh=rows)
s2= figure(title="VNIR PCA1", width=cols, height=rows, x_range=s1.x_range, y_range=s1.y_range)
s2.axis.visible = False
s2.image(image=[np.flipud(pca)], x=[0], y=[0], dw=[cols], dh=[rows])

p = gridplot([[s1], [s2]], toolbar_location='above')
show(p)

Look that the brightness patters seems to overal match, which will help SIFT+FLANN to find the matheces.
For instance,  within the grapevine that the brightest vines in the the LST (warmer temperatures) match with the lowest brighness in the PCA, these areas were under a more stressed deficit irrigation and thus both temperatures. However, some other areas have opposite brighness, such as the calibration pannels placed on top of the scenes, the warmest pannels (brightest in LST) are however the darkest (black painted) in the VNIR PCA1.

## Run SIFT + FLANN
Now that we have the data preprocess, we can run our SIFT and FLANN algorithms to find common features that we will be considered as potential GCPs. For this task we will use the powerful [OpenvCV] library, available in Python[^2], which, among many others, contains SIFT and FLANN methods:

[^2]: also in C++ and Java

In [ ]:
# Import Python OpenCV
import cv2
# matching factor between keypoint descriptor
match_factor = 0.80 

# SIFT and FLANN needs to deal with 8bit images, so we need to rescale the floaging point input to 0-255 bit range
master_scaled = img.scale_grayscale_image(pca, no_data=np.nan)
slave_scaled = img.scale_grayscale_image(lst, no_data=np.nan)

# Get the LST (slave) GDAL geotransform and projection
fid = gdal.Open(lst_file, gdal.GA_ReadOnly)
slave_geo = fid.GetGeoTransform()
proj = fid.GetProjection()
del fid
# Get the VNIR (master) GDAL geotransform and projection
fid = gdal.Open(pca_image, gdal.GA_ReadOnly)
master_geo = fid.GetGeoTransform()
proj = fid.GetProjection()
del fid

# Initiate SIFT detector
detector = cv2.SIFT_create()
# We use NORM distance measurement for SIFT
norm_type = cv2.NORM_L1

print("Finding features and their descriptors, this might take a while...")
kp_master, des_master = detector.detectAndCompute(master_scaled, None)
kp_slave, des_slave = detector.detectAndCompute(slave_scaled, None)

# Find matches between slave and master descriptors
matcher = cv2.BFMatcher(norm_type)
# Get the 2 best matches per feature
matches = matcher.knnMatch(des_master, des_slave, k=2)

print(f"Found {len(matches)}, filtering by FLANN factor similarity of {match_factor}")
# Create a list of potential GCPs
gcp_list = []
good_matches = []
for i, (m, n) in enumerate(matches):
    # Only the most similar matches, based on FLANN match factor, are selected
    if m.distance < match_factor * n.distance:
        good_matches.append(m)
        master_pt = np.float32(kp_master[m.queryIdx].pt)
        # Get the projected map coordinates (X, Y) of the master
        x_master, y_master = img.get_map_coordinates(float(master_pt[1]),
                                                     float(master_pt[0]),
                                                     master_geo)
        # Get the image coordinates (row, col) of the slave
        slave_pt = np.float32(kp_slave[m.trainIdx].pt)
        gcp_list.append((x_master,
                         y_master,
                         float(slave_pt[1]),
                         float(slave_pt[0])))

print(f"Found {len(gcp_list)} potential GCPs")

### Visualize the potential GCP
We can now visualize all the matches found by SIFT+FLANN, according to the resemblance ratio we have just set:

In [ ]:
from matplotlib import pyplot as plt
#-- Draw matches
img_matches = np.empty((max(slave_scaled.shape[0], master_scaled.shape[0]), 
                        slave_scaled.shape[1] + master_scaled.shape[1], 3), 
                       dtype=np.uint8)
cv2.drawMatches(master_scaled, kp_master, slave_scaled, kp_slave, good_matches, img_matches,
                matchesThickness=10, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#-- Show detected matches
plt.imshow(img_matches)
plt.show()

## Filter GCP by translation distance
Since we assume that a priori both orthomosaics were georreferenced, either the UAV avionics information, the use of in situ GCP, or both, we can consider that the displacment between the `slave` and the `master` should not be too large, and thus we could discard GCPs which translation vector is larger than a given distance

:::{hint}
For instance we can assume that the UAV GPS could have an unsistematic error of 10m and thus the displacement between mosaics should not be larger than this distance. 

Or we can just browse both mosaics and evaluate the observed maximum displacement between mosaics
:::

In [ ]:
# Set a maximum translation distance between slave and master
dist_threshold = 10

# Convert the list of potential GCPs to a numpy array to make computations more effective
gcp_list = np.asarray(gcp_list)

if len(gcp_list.shape) == 1:
    gcp_list = gcp_list.reshape(1, -1)

# Get the GCP map coordinates (X, Y) from the slave image coordinates
x_slave, y_slave = img.get_map_coordinates(gcp_list[:, 2], gcp_list[:, 3], slave_geo)
# Compute the the distance between the slave map coordinates and the master map coordinates for each GCP
dist = np.sqrt((x_slave - gcp_list[:, 0]) ** 2 + (y_slave - gcp_list[:, 1]) ** 2)
# Keep only those GCPs whose distance is lower than the threshold we set
gcp_list = gcp_list[dist <= dist_threshold]
# Convert back the list of GPC to a Python list
gcp_list = gcp_list.tolist()
good_matches = np.array(good_matches)[dist <= dist_threshold].tolist()
print(f'Got {len(gcp_list)} valid GPCs with a translation lower than {dist_threshold}m')

### Visualize the potential GCP filtered by distance

In [ ]:
cv2.drawMatches(master_scaled, kp_master, slave_scaled, kp_slave, good_matches, img_matches,
                matchesThickness=10, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#-- Show detected matches
plt.imshow(img_matches)
plt.show()

:::{seealso}
:class:dropdown
You can check [this function in the GitHub source code](https://github.com/hectornieto/airborne_tools/blob/5db17192e638c2745dea5d918b9dcaffd05a14cf/airborne_tools/gcp_tools.py#L568)
:::

## Remove GCPs that are too close each other
On the other hand, we wanted to have well distributed GCPs across the whole image. Theferore we will discard GCPs that were very close to other GCPs, aiming to avoid overrepresnting one area over others:

In [ ]:
# Set minimum distance in pixels that GCPs should be from other GCPs
pixel_threshold = 5

gcps_good = []
matches_good = []
for i, gcp_test in enumerate(gcp_list):
    good = True
    if i == len(gcp_list) - 2:
        continue
    for j in range(i + 1, len(gcp_list)):
        dist = np.sqrt((gcp_test[2] - gcp_list[j][2]) ** 2 + (gcp_test[3] - gcp_list[j][3]) ** 2)
        if dist < pixel_threshold:  # GCPs closer to each other are discarded to avoid overfitting
            good = False
            break
    if good:
        gcps_good.append(gcp_test)
        matches_good.append(good_matches[i])

gcp_list = list(gcps_good)
good_matches = list(matches_good)
print(f'Got {len(gcp_list)} valid GPCs far enough from each other by a distance of {dist_threshold} pixels')

### Visualize the filtered GCP

In [ ]:
cv2.drawMatches(master_scaled, kp_master, slave_scaled, kp_slave, good_matches, img_matches,
                matchesThickness=10, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#-- Show detected matches
plt.imshow(img_matches)
plt.show()

:::{seealso}
:class:dropdown
You can check [this function in the GitHub source code](https://github.com/hectornieto/airborne_tools/blob/5db17192e638c2745dea5d918b9dcaffd05a14cf/airborne_tools/gcp_tools.py#L594)
:::

## Remove GCPs that are outliers in a 3rd degree polynomial warp
To finalize, we will discard all the GCPs that are outliers after fitting at 3rd degree polynomial warp

In [ ]:
# Set a warping error threshold of 2.5 cm
warp_threshold = 0.025

# Convert the list of GPCs to a python numpy array to vectorize the calculations
gcps = np.asarray(gcp_list)
good_matches = np.asarray(good_matches)

# Create our 3rd degree polynomial warping helper function
def _fit_polynomial_warp(gcps):

    gcps = np.asarray(gcps)
    if gcps.shape[0] < 15:
        return None, None
    rows = gcps[:, 2]
    cols = gcps[:, 3]
    rows2 = rows ** 2
    cols2 = cols ** 2
    rowscols = rows * cols
    rows2cols = rows ** 2 * cols
    rowscols2 = rows * cols ** 2
    rows3 = rows ** 3
    cols3 = cols ** 3

    x = np.matrix([np.ones(rows.shape), rows, cols, rowscols, rows2, cols2, rows2cols, rowscols2, rows3, cols3]).T
    map_x = gcps[:, 0].reshape(-1, 1)
    map_y = gcps[:, 1].reshape(-1, 1)
    theta_x = (x.T * x).I * x.T * map_x
    theta_y = (x.T * x).I * x.T * map_y
    return np.asarray(theta_x).reshape(-1), np.asarray(theta_y).reshape(-1)


# Create our helper function to compute the warping error for each GCP
def _calc_warp_erors(gcps, theta_x, theta_y):
    def _polynomial_warp(rows, cols, theta_x, theta_y):
        x = theta_x[0] + theta_x[1] * rows + theta_x[2] * cols + theta_x[3] * rows * cols + theta_x[4] * rows ** 2 + \
            theta_x[5] * cols ** 2 + theta_x[6] * rows ** 2 * cols + theta_x[7] * rows * cols ** 2 + \
            theta_x[8] * rows ** 3 + theta_x[9] * cols ** 3
        y = theta_y[0] + theta_y[1] * rows + theta_y[2] * cols + theta_y[3] * rows * cols + theta_y[4] * rows ** 2 + \
            theta_y[5] * cols ** 2 + theta_y[6] * rows ** 2 * cols + theta_y[7] * rows * cols ** 2 + \
            theta_y[8] * rows ** 3 + theta_y[9] * cols ** 3
        return x, y

    gcps = np.asarray(gcps)
    if len(gcps.shape) == 1:
        gcps = gcps.reshape(1, -1)

    rows = gcps[:, 2]
    cols = gcps[:, 3]
    x_model, y_model = _polynomial_warp(rows, cols, theta_x, theta_y)
    error = np.sqrt((x_model - gcps[:, 0]) ** 2 + (y_model - gcps[:, 1]) ** 2)
    return error

# FIt a 3rd order polynomial warp
theta_x, theta_y = _fit_polynomial_warp(gcps)
error = _calc_warp_erors(gcps, theta_x, theta_y)
while np.max(error) > warp_threshold and len(error) > 30:
    index = error.argsort()[::-1]
    gcps = gcps[index[1:]]
    good_matches = good_matches[index[1:]]
    theta_x, theta_y = _fit_polynomial_warp(gcps)
    error = _calc_warp_erors(gcps, theta_x, theta_y)

gcp_valid = list(gcps)
good_matches = list(good_matches)
print(f'Got {len(gcp_valid)} valid GPCs that fit well in a 3rd degree polynomial warp')

### Visualize the filtered GCPs

In [ ]:
cv2.drawMatches(master_scaled, kp_master, slave_scaled, kp_slave, good_matches, img_matches,
                matchesThickness=10, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#-- Show detected matches
plt.imshow(img_matches)
plt.show()

:::{seealso}
:class:dropdown
You can check [this function in the GitHub source code](https://github.com/hectornieto/airborne_tools/blob/5db17192e638c2745dea5d918b9dcaffd05a14cf/airborne_tools/gcp_tools.py#L761)
:::


## Create and save the new collocated `slave` image
After running the cell below a new image will be saved in the `/input/UAV` folder, with suffix `_collocated.tif`. You can visualize the results on QGIS. In addition, in the `/input/UAV/GCS` you will have access to an ASCII table that summarizes the automatic GCPs found, and a line shapefile that represents the traslation vector from the original `slave` GCP coordinate to its corresponding coordinate in the `reference`.

In [ ]:
from airborne_tools import gcp_tools as gcp
# Set transformation method
# 0 for thin plate spline transformation, 
# 1 for 1st degree polynomial
# 2 for 2nd degree polynomial
# ...
transform = 0  

# Set the extent for the output image equal as the master image
xmin, ymax = img.get_map_coordinates(0, 0, master_geo)
xmax, ymin = img.get_map_coordinates(master_scaled.shape[1], master_scaled.shape[0], master_geo)
output_extent = (xmin, ymin, xmax, ymax)

collocated_image = test_dir / 'tir_odm_20220916_collocated.tif'

# Get the origial GCP map coordinates
slave_xcoord, slave_yCoord = img.get_map_coordinates(np.asarray(gcp_valid)[:, 2],
                                                     np.asarray(gcp_valid)[:, 3],
                                                     slave_geo)

# Create an anciillary GCP subfolder to store the final GPCs
if not (test_dir / "GCPs").is_dir():
    (test_dir / "GCPs").mkdir(parents=True)

# Save the transformation line vector betwen the original coordinates and the collocated coordiantes
outshapefile = test_dir / 'GCPs' / f"{collocated_image.name[:-4]}_Transform.shp"
gcp._write_transformation_vector((slave_xcoord, slave_yCoord),
                                 (np.asarray(gcp_valid)[:, 0], np.asarray(gcp_valid)[:, 1]),
                                 outshapefile,
                                 proj)

# Write the GCP to ascii file
outtxtfile = test_dir / 'GCPs' / f"{collocated_image.name[:-4]}_GCPs.txt"
gcp.gcps_to_ascii(gcp_valid, outtxtfile)

# Finally use a airbone_tools helper function to perform the reprojection with the GCPs
gcp.warp_image_with_gcps(lst_file,
                         gcp_valid,
                         collocated_image,
                         output_extent=None,
                         src_no_data=lst_no_data,
                         transform=transform,
                         data_type=gdal.GDT_UInt16)

print(f"Saved collocated image in f{collocated_image}")

:::{seealso}
:class: dropdown
You can check the full code in the [airborne_tools GitHub source code repository](https://github.com/hectornieto/airborne_tools/blob/5db17192e638c2745dea5d918b9dcaffd05a14cf/airborne_tools/gcp_tools.py#L18)
:::

:::{hint}

Have a look in QGIS at the collocated TIR image comapred to the uncorrected TIR and Sequoia Multispectral image to visualize the colocation change. 
:::

# Conclusions

* We introduced an automatic method to corregister two mosaics
* This method does not require that both mosaics share the same resolution nor the same extension, as SIFT tries to find scale-invariant features in the images.
* After running SIFT+FLANN we run serveral filter criteria in order to use only the best GCPs.
* [...]

:::{note}
Please feel free to comment any thoughts. This is work in progress!!!
:::